In [ ]:
class Graph:
  def __init__(self):
    self.graph = {} 
  
  def add_vertex(self, label):
    if label not in self.graph.keys():
      self.graph[label] = [] 

  def has_edge(self, v1, v2):
    return v2 in self.graph[v1]

  def add_edge(self, v1, v2, directed = True):
    self.add_vertex(v1)
    self.add_vertex(v2)

    if(not directed and not self.has_edge(v2,v1)):
        self.graph[v2].append(v1)
        return
    if(not self.has_edge(v1,v2)):
      self.graph[v1].append(v2)
    if(not self.has_edge(v2,v1)):
      self.graph[v2].append(v1)

  def print_graph(self):
    print(self.graph)

  def DFS(self, u, vis=[]):
    if u not in vis: 
      vis.append(u) 
      for i in self.graph[u]: 
        if i not in vis: 
          vis.append(i) 
          self.DFS(i, vis) 
      return vis

def limpiar_diccionario(dictio:dict):
  toTakeAout=[]
  for key in dictio:
    if not dictio[key]:
      toTakeAout.append(key)
    elif not dictio[key][0]:
      toTakeAout.append(key)
    elif not dictio[key][1]:
      toTakeAout.append(key)
    elif  "Category" in dictio[key][1]:
      toTakeAout.append(key)
    elif ":" in dictio[key][1]:
      toTakeAout.append(key)
  for key in toTakeAout:
     dictio.pop(key)
  return dictio
def filtrar_personas_a(etiquetas,fuenteTexto, analizadorTexto, rutaActual):
    elementos = {}
    text = ""
    for etiqueta in etiquetas:
      elementos.update({etiqueta.text:[etiqueta.text,etiqueta.get("href")]}) 
    for parrafo in fuenteTexto:
      text = f"{text} {parrafo.text}" 
    analizado = analizadorTexto(text)  
    filtrados = {}
    for elemento in analizado.ents:
        if(elemento.label_ == "PERSON" and elemento.text not in rutaActual):
            filtrados.update({elemento.text:elementos.get(elemento.text)}) 
    filtrados = limpiar_diccionario(filtrados)
    return filtrados

def AumentarConexiones(nuevosNodos, grafo, nodoActual): 
  for nodo in nuevosNodos:
      if(nodo not in grafo.graph.keys()):
        grafo.add_vertex(nodo)
      if nodo not in grafo.graph[nodoActual]:
        grafo.add_edge(nodoActual, nodo)


import  requests
from bs4 import BeautifulSoup
import spacy

def buscar_ruta(urlInicial=False):
  grafo = Graph()
  urlActual =""
  RutaActual = []
  if not urlInicial:
    personaInicial = input("Ingrese P1: ")
    nombreLista= personaInicial.split(" ")
    nombreFormaCorrecta = "_".join(nombreLista)
    urlActual = "https://en.wikipedia.org//wiki/" + nombreFormaCorrecta
    
  else:
    personaUrlInicial = input("Ingrese url de P1: ")
    urlActual = personaUrlInicial
    listaUrl = personaUrlInicial.split("/")
    nombre= listaUrl[-1]
    personaInicial = nombre.replace("_"," ")
  RutaActual.append(personaInicial)
  grafo.add_vertex(personaInicial)
  personaFinal = input("Ingrese P2: ")
  analizadorTexto = spacy.load("en_core_web_sm")
  found = False
  personasPasadas = 0
  while not found and personasPasadas<50:
    personasPasadas +=1
    RESPUESTA = requests.get(urlActual)
    html = RESPUESTA.text
    soup = BeautifulSoup(html, "html.parser")  
    etiquetas_a = soup.find_all("a")
    etiquetas_p=soup.find_all("p",limit=20)
    etiquetas_filtradas_dict = filtrar_personas_a(etiquetas_a,etiquetas_p,analizadorTexto, RutaActual)
    AumentarConexiones(etiquetas_filtradas_dict, grafo, personaInicial)
    if personaFinal in etiquetas_filtradas_dict.keys(): 
      RutaActual.append(personaFinal)
      found = True 
      break
    if len(etiquetas_filtradas_dict )>0:
        key= next(iter(etiquetas_filtradas_dict)) 
        personaInicial = etiquetas_filtradas_dict.get(key)[0]
        urlActual = "https://en.wikipedia.org/" + etiquetas_filtradas_dict.get(key)[1]
        RutaActual.append(personaInicial) 
    else:
      print("Sin personas en que buscar, interrupcion adelantada")
      break
  if not found:
    print("no están conectados", RutaActual)
  else:
    print("Estan conectados, la ruta fue",RutaActual)
  return grafo,RutaActual
grafos = []
rutas= []
for x in range(3):
  grafo,ruta= buscar_ruta()
  grafos.append(grafo)
  rutas.append(ruta)
for i in range(len(rutas)):
  print(rutas[i])
  grafos[i].print_graph()


Ingrese P1: james rodríguez 
Ingrese P2: falcao garcía 
Sin personas en que buscar, interrupcion adelantada
no están conectados ['james rodríguez ']
